In [1]:
import pandas as pd
import io
import msoffcrypto as ex
import numpy as np
import warnings
import re
global counta, countb, valsa, valsb, df, dfmerge

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 20)

HL7Field = ['MSH','PID','ORC','OBR','OBX','SPM','NTE','IN1','IN2','DG1','0009']
filedir = r"C:\Users\gmoye\FlatFileHL7\Extract_for_COVID_and_Influenza_20230503_1151.xlsx"
mapdir = r"C:\Users\gmoye\FlatFileHL7\FieldMap.xlsx"
hl7map = r"C:\Users\gmoye\FlatFileHL7\Maps.xlsx"
hometestmap = r"C:\Users\gmoye\FlatFileHL7\COVID-At-Home-Test-HL7-Guide-5-4-2022_0.xlsx"
RequiredFields = r"C:\Users\gmoye\FlatFileHL7\RequiredFields.xlsx"
Separators = r"C:\Users\gmoye\FlatFileHL7\Separators.xlsx"

pw = "test"

with warnings.catch_warnings():
    warnings.simplefilter('ignore', UserWarning)
    warnings.simplefilter('ignore', pd.errors.PerformanceWarning)
    warnings.simplefilter('ignore', FutureWarning)

    decrypted_workbook = io.BytesIO()
    with open(filedir, 'rb') as file:
        office_file = ex.OfficeFile(file)
        office_file.load_key(password=pw)
        office_file.decrypt(decrypted_workbook)
    
    all_sheets = pd.read_excel(decrypted_workbook, sheet_name=None)
    sheets = list(all_sheets.keys())

    df = pd.read_excel(decrypted_workbook, sheet_name=sheets[0], parse_dates=[['Order Date', 'Order Time'],['Result Date','Result Time'],['Collection Date','Collection Time']])
    
    count = 0
    for column, data in df.iteritems():
        count += 1
    
    df['record_no'] = np.arange(len(df))+1
    rcount = df.pop('record_no')
    df.insert(0, rcount.name, rcount)
    
    count = 0
    for col, vals in df[['Last Lab Results','Test Details']].items():
        count+=1
        count2 = 0
        for val in vals:
            count2+=1
            val = "'" + val + "'"
            val = "','".join(val.splitlines())
            val = str(val)
            df[f'{col}'].loc[df['record_no'] == count2] = val
    
    df2 = df.loc[((df['Last Lab Results'].str.contains(',')) | (df['Test Details'].str.contains(',')))]
    df = df.loc[~((df['Last Lab Results'].str.contains(',')) | (df['Test Details'].str.contains(',')))]
    
    df = df.drop(columns=['record_no'])
    df2 = df2.drop(columns=['record_no'])
    
    df2preserve = df2.copy(deep=True)
    
    df2 = df2[['Order ID', 'Last Lab Results','Test Details']]
    
    df2['record_no'] = np.arange(len(df2))+1
    rcount = df2.pop('record_no')
    df2.insert(0, rcount.name, rcount)
    
    for val in df2['record_no']:
        df2['record_no'].loc[df2['record_no'] == val] = str(val)+','+str(val)
    
    df3 = (df2.set_index(['Order ID'])).apply(lambda x: x.str.split(',').explode(['record_no','Last Lab Results','Test Details']))
    df2['record_no'] = df2['record_no'].str.replace(r'\,\d','').astype(object)
    
    dfsep = pd.merge(df2, df3, on = ['record_no'], how = 'inner')
    dfsep = dfsep.drop(columns=['Last Lab Results_x','Test Details_x'])
    df2preserve = df2preserve.drop(columns=['Last Lab Results','Test Details'])
    dfsep = dfsep.rename(columns={'Last Lab Results_y':'Last Lab Results','Test Details_y':'Test Details'})
    dfsep = pd.merge(dfsep, df2preserve, on = ['Order ID'], how = 'outer')


    df = pd.concat([df, dfsep]).reset_index(drop = True)

    df['Patient_ID_assigner'] = 'EPIC.ACC'
    df['Patient_ID_2_assigner'] = 'EPIC.MRN'
    df = df.rename(columns={'Order Date_Order Time':'Order_test_date','Result Date_Result Time':'Test_date','Collection Date_Collection Time':'Specimen_collection_date_time','Employment Status':'Employment_status'})
    df = df.drop(columns=['record_no'])
    df.index.names = ['Field']

    #display(df)
    
    column_names = df.columns.values.tolist()
    dfcolumns = pd.DataFrame(column_names)
    dfcolumns.rename(columns={0:'Field'}, inplace=True)

    dfmappings = pd.read_excel(mapdir)

    dfmappings = dfmappings.loc[dfmappings['Not Needed'] != 'X']

    dfdistinctmappings = dfmappings[['Field','NatELR Field ID', 'Segment', 'Not Needed','Option-Set needed']]

    dfmerge = pd.merge(dfcolumns, dfdistinctmappings, on = ['Field'], how = 'outer')
    dfmerge['NatELR Field ID'].loc[dfmerge['NatELR Field ID'].isna()] = dfmerge['Field']
    
    for col, val in dfmerge[['Field']].items():
        counta =-1
        for valsa in val:
            counta += 1
            valsa = str(valsa)
            for col, val in dfmerge[['NatELR Field ID']].items():
                countb = -1
                for valsb in val:
                    countb +=1
                    valsb = str(valsb)
                    if counta == countb:
                        df = df.rename(columns={f'{valsa}':f'{valsb}'}) 
        
    dfhl7 = pd.read_excel(hl7map)
    
    dfhl7home = pd.read_excel(hometestmap)
    
    dfhl7Separators = pd.read_excel(Separators)
    
    dflinesegments = pd.DataFrame(columns = ['Line Name','Line Code','Main Code','Sub Code','Segment', 'Field Name'])
    
    for vals in dfhl7Separators.values:
        linesubcode = None
        vals = vals[0]
        vals = vals.replace('-', '', 1).replace('.', '-', 1)
        Segment = vals.split(' ', 1)
        Segment1 = Segment[0]
        sep = '-'
        sep1 = '.'
        LineName = Segment1.split(sep, 1)[0]
        LineCode = Segment1.split(sep, 1)[1]
        CodeSplit = LineCode.split(sep1, 1)
        if len(CodeSplit) > 1:
            LineMainCode = int(CodeSplit[0])
            LineSubCode = int(CodeSplit[1])
        else:
            LineMainCode = int(CodeSplit[0])
            LineSubCode = '0'
            
        Segment2 = Segment[1].replace(' ', '', 1)
        count +=1
        dflinesegments = dflinesegments.append({'Line Name': LineName, 'Line Code': LineCode, 'Main Code': LineMainCode, 'Sub Code': LineSubCode, 'Segment': Segment1, 'Field Name': Segment2}, ignore_index=True)    
    
    dflinesegments = dflinesegments.sort_values(['Line Name', 'Main Code', 'Sub Code'], ascending=[1,1,1])
    
    pipe = '|'
    carrot = '^'
    dflinesegments['Separator'] = ''
    dflinesegments.loc[dflinesegments['Sub Code'] == '0', 'Separator'] = pipe
    dflinesegments.loc[dflinesegments['Sub Code'] != '0', 'Separator'] = carrot
    
    HL7Line = dfhl7home['Segment'].unique()
    
    dfhl7RequiredFields = pd.read_excel(RequiredFields)
    dfhl7Meaning = dfhl7RequiredFields.loc[dfhl7RequiredFields['HL7 Segment'].isna()]
    dfhl7Meaning = dfhl7Meaning.drop(dfhl7Meaning.columns[[1,2]], axis = 1).reset_index(drop = True)
    dfhl7RequiredFields = dfhl7RequiredFields.loc[dfhl7RequiredFields['HL7 Segment'].isna() == False]
    dfhl7RequiredFields = dfhl7RequiredFields.reset_index(drop = True)
    dfhl7RequiredFields['HL7 Element'] = dfhl7RequiredFields['HL7 Segment'].replace(regex=r"-\d{1,2}",value='').replace(regex=r"\.(.*)",value='')
    field = dfhl7RequiredFields.pop('HL7 Element')
    dfhl7RequiredFields.insert(1, field.name, field)
    
    dfhl7pivot = dfhl7.pivot(index=['Variable Name','Instructions'], columns=['Hardcode'], values=['HL7 Field'])#.reset_index(drop=False)
    dfhl7pivot.columns = dfhl7pivot.columns.droplevel(0)
    dfhl7pivot = dfhl7pivot.dropna(axis=1, how='all')
    dfhl7pivottest = dfhl7pivot.reset_index()
    dfhl7pivotreduction = dfhl7pivot.loc[:, dfhl7pivot.columns.notna()].dropna(axis=0, how='all').reset_index(drop=False)
    dfhl7pivotreduction["Variable Name"] = dfhl7pivotreduction["Variable Name"].fillna(dfhl7pivotreduction["Instructions"])
    dfhl7pivotreduction = dfhl7pivotreduction.set_index(["Instructions", "Variable Name"]).T#.reset_index(drop=False)
    dfhl7pivotreduction.columns = dfhl7pivotreduction.columns.droplevel(0)
    dfhl7pivotreduction = dfhl7pivotreduction.reset_index(drop=False)
    
    lcol = list(dfhl7pivotreduction.columns)
    lcol.remove('Hardcode')
    
    vallist3 = []
    vallist4 = []
    vallist5 = []
    
    for val in dflinesegments['Line Name']:
        vallist3.append(val)
    for val in dflinesegments['Separator']:
        vallist4.append(val)
    for val in dflinesegments['Segment']:
        vallist5.append(val)
    
    listvals = dfmappings['NatELR Field ID'].values.tolist()
    listvals2 = dfmappings['Field Match - Short'].values.tolist()
    counta = -1
    for vals1 in listvals:
        counta += 1
        countb = -1
        for vals2 in listvals2:
            countb += 1
            if counta == countb:
                dfhl7RequiredFields.loc[dfhl7RequiredFields['HL7 Element Name'] == vals2, 'HL7 Nat ELR Match'] = vals1
    
    field = dfhl7RequiredFields.pop('HL7 Nat ELR Match')
    dfhl7RequiredFields.insert(1, field.name, field)
    
    loops = -1
    
    dfhl7RequiredFields = dfhl7RequiredFields.rename(columns={'HL7 Segment':'Segment'})
    #display(dfhl7RequiredFields)
    #display(dflinesegments)
    
    dfforloopsfordays = pd.merge(dflinesegments, dfhl7RequiredFields, on = ['Segment'], how = 'left')
    dfforloopsfordays = dfforloopsfordays.rename(columns={'Separator_x':'Separator'})
    dfforloopsfordays['HL7 Element Name'] = dfforloopsfordays['HL7 Element Name'].fillna(dfforloopsfordays['Field Name'])
    dfforloopsfordays = dfforloopsfordays.drop(columns=['HL7 Element Name'])
    dfforloopsfordays = dfforloopsfordays.astype({'Main Code': int, 'Sub Code': int})
    dfforloopsfordays = dfforloopsfordays.sort_values(['Line Name','Main Code','Sub Code'], ascending=[1,1,1]).reset_index(drop = True)
    dfforloopsfordays = dfforloopsfordays.drop_duplicates(subset = 'Segment', keep = 'first')
    #display(dfhl7pivotreduction)
    pd.set_option('display.max_columns', None)
    pd.options.mode.chained_assignment = None
    pd.set_option('display.max_rows', None)
    #display(dfforloopsfordays)
        
    dfhl7RequiredFieldsMerge = pd.merge(dflinesegments, dfhl7RequiredFields, on = ['Segment'], how = 'outer')
    dfhl7RequiredFieldsMerge['HL7 Nat ELR Match'] = dfhl7RequiredFieldsMerge['HL7 Nat ELR Match'].fillna(dfhl7RequiredFieldsMerge['Field Name'])
    
    #dfsep = pd.merge(df2, df3, on = ['record_no'], how = 'inner')
        
    dfpivredlist = dfhl7pivotreduction.columns.to_list()
        
    #print(dfpivredlist, '\n')
    for val in dfpivredlist:
        if val != 'Hardcode':
            for vals in dfhl7pivotreduction[f'{val}']:
                vals = str(vals)
                if vals != 'nan':
                    dfhl7pivotreduction = dfhl7pivotreduction.rename(columns={f'{val}':f'{vals}'})
    
    dfpivredlist = dfhl7pivotreduction.columns.to_list()
    HC = pd.DataFrame()
    
    rem = []
    add = []
    for val in enumerate(dfpivredlist):
        #print(val[1])
        val = str(val[1])
        if ' ' in val:
            rem.append(val)
            #dfpivredlist.remove(val)
            val = val.replace(' ', '')
            pattern = r'\[.*?\]'
            val = re.sub(pattern, '', val)
            #if val == 'PID-3.4.1':
                #val = 'PID-3.4'
            add.append(val)
           
    for vals in rem:
        dfpivredlist.remove(vals)
      
    for vals in add:
        dfpivredlist.append(vals)
    
    #print(dfpivredlist, '\n')
    dfpivredlist = [*set(dfpivredlist)]
    #print(dfpivredlist, '\n')
    
    dfhl7pivotreduction = dfhl7pivotreduction.replace('PID-3.4.1 [1]', 'PID-3.4').replace('PID-3.4.1 [2]', 'PID-3.4')
    dfhl7pivotreduction = dfhl7pivotreduction.replace('PID-3.5 [1]', 'PID-3.5').replace('PID-3.5 [2]', 'PID-3.5')
    dfhl7pivotreduction = dfhl7pivotreduction.rename(columns={'PID-3.4.1 [1]':'PID-3.4','PID-3.4.1 [2]':'PID-3.4'})
    dfhl7pivotreduction = dfhl7pivotreduction.rename(columns={'PID-3.5 [1]':'PID-3.5','PID-3.5 [2]':'PID-3.5'})
    dfhl7RequiredFieldsMerge['Segment'] = dfhl7RequiredFieldsMerge['Segment'].str.strip()
    #display(dfhl7pivotreduction)
    #display(len(dfhl7pivotreduction.columns))
    hardlist = []
    counta = -1
    for vals in dfpivredlist:
        counta += 1
        if vals not in ['Hardcode','PID-3.4','PID-3.4.1','PID-3.5']:
            vals = str(vals)
            #print(vals)
            strcount = str(counta)
            hardval = str('HC'+strcount)
            hardlist.append(str(hardval))
            hardval = dfhl7pivotreduction.loc[dfhl7pivotreduction[f'{vals}'] == f'{vals}']
            #print(hardval)
            hardval = hardval.dropna(axis=1, how='all')
            hardval = hardval.reset_index(inplace = False, drop = True)
            globals()[str('HC'+strcount)] = hardval
    print('\n')
            #display(hardval)
            
        #df2['record_no'] = df2['record_no'].str.replace(r'\,\d','').astype(object)
    #print(hardlist)
    
    dfhl7RequiredFieldsMerge['Hardcode'] = None
    dfhl7RequiredFieldsMerge.loc[dfhl7RequiredFieldsMerge['Segment'] == 'PID-3.4', 'Hardcode'] = 'EPIC.ACC or EPIC.MRN'
    dfhl7RequiredFieldsMerge.loc[dfhl7RequiredFieldsMerge['Segment'] == 'PID-3.5', 'Hardcode'] = 'AN or MR'
    #dfforloopsfordays['Segment'] = str(dfforloopsfordays['Segment'])
    #dfforloopsfordays['Hardcode'] = dfforloopsfordays['Hardcode'].astype(float)

    for val in hardlist:
        global values
        values = []
        dfs = eval(val)
        #display(dfs)
        for (column, data) in dfs.iteritems():
            if column == 'Hardcode':
                HC = data.values[0]
        for (column, data) in dfs.iteritems():
            if column != 'Hardcode':
                value = data.values[0]
                values.append(value)
        for vals in values:
            dfhl7RequiredFieldsMerge.loc[dfhl7RequiredFieldsMerge['Segment'] == f'{vals}', 'Hardcode'] = HC
        #print(HC)
        #print(values)
    
    dfhl7RequiredFieldsMerge = dfhl7RequiredFieldsMerge.drop(columns = ['Field Name','HL7 Element Name','Separator_y','HL7 Data Type','Data Type','Order'])
    
    dfhl7RequiredFieldsMerge.loc[(dfhl7RequiredFieldsMerge['Segment'] == 'MSH-4') & (dfhl7RequiredFieldsMerge['HL7 Nat ELR Match'] == 'Sending Facility'), 'HL7 Nat ELR Match'] = 'Testing_lab_name'
    dfhl7RequiredFieldsMerge.loc[(dfhl7RequiredFieldsMerge['Segment'] == 'OBR-22') & (dfhl7RequiredFieldsMerge['HL7 Nat ELR Match'] == 'Results Rpt/Status Chng - Date/Time'), 'HL7 Nat ELR Match'] = 'Test_date'
    dfhl7RequiredFieldsMerge.loc[(dfhl7RequiredFieldsMerge['Segment'] == 'OBR-4') & (dfhl7RequiredFieldsMerge['HL7 Nat ELR Match'] == 'Universal Service Identifier'), 'HL7 Nat ELR Match'] = 'Ordered_test_code'
    dfhl7RequiredFieldsMerge.loc[(dfhl7RequiredFieldsMerge['Segment'] == 'SPM-17') & (dfhl7RequiredFieldsMerge['HL7 Nat ELR Match'] == 'Specimen Collection Date/Time'), 'HL7 Nat ELR Match'] = 'Specimen_collection_date_time'
    dfhl7RequiredFieldsMerge.loc[(dfhl7RequiredFieldsMerge['Segment'] == 'IN1-42') & (dfhl7RequiredFieldsMerge['HL7 Nat ELR Match'] == "Insured's Employment Status"), 'HL7 Nat ELR Match'] = 'Employment_status'

    #display(dfhl7RequiredFieldsMerge)
    #display(dfhl7pivotreduction)
    #display(dfforloopsfordays)
    
    #print(df.columns)

    display(dfhl7RequiredFieldsMerge)
        
    count = 0
    p1 = 'linesegment'
    varlist = []
    countalpha = -1
    for i in range(len(df)):
        countalpha += 1
        strcountalpha = str(countalpha)
        count4 = 0
        for val in HL7Field:
            linesegment = val
            inc2 = val+'_'+strcountalpha
            segval = str(p1 + inc2)
            varlist.append(str(segval))
            if val != '0009':
                line = f'{val}' + "|"
            else:
                line = f'{val}'
            count += 1
            count1 = 0
            for field, val1 in dfhl7RequiredFieldsMerge[['HL7 Element','Segment','HL7 Nat ELR Match']].items():
                count1 += 1
                if count1 == 2:
                    count2 = -1
                    for val2 in val1:
                        #print(count2)
                        count2 += 1
                        for value in dfhl7RequiredFieldsMerge['HL7 Nat ELR Match'].items():
                            f = dfhl7RequiredFieldsMerge['HL7 Nat ELR Match'].values[count2]
                            #print(f, val2)
                            break
                        val2 = str(val2)
                        if val in val2:
                            try:
                                #print(val2, val)
                                Hardcode = dfhl7pivotreduction['Hardcode'].loc[(dfhl7pivotreduction.eq(f'{val2}').any(axis=1)) & (dfhl7pivotreduction['Hardcode'] != '|')]
                                #Hardcode = str(Hardcode)
                                #display(Hardcode)
                                Hardcode = Hardcode.item()
                                if count2 == 1:
                                    if Hardcode != []:
                                        line += (Hardcode + '')
                                    else:
                                        continue
                                if count2 > 2:
                                    if count2 in [754,760,915,1108,1121,1758]:
                                        pass
                                    #print(field)
                                    else:
                                        if Hardcode != []:
                                            line += (Hardcode + vallist4[count2] + '')
                                        else:
                                            continue
                                else:
                                    continue
                            except:
                                pass
                            
                            count4 = -1
                            count5 = 0
                            for i in range(len(df)):
                                count4 += 1
                                for fieldval in df.columns:
                                    #count4 += 1
                                    fieldval = str(fieldval)
                                    f = str(f)
                                    for i in range(len(df)):
                                        #count4 += 1
                                        #print(i)
                                        if f == fieldval:
                                            #print(inc,count5)
                                            v = df.loc[countalpha, fieldval]
                                            v = str(v)
                                            #print(count4, fieldval, f': {v}', val, val2)
                                            line += (vallist4[count2] + v + '')
                                        break
                                    count5 += 1
                                break
                                #break
                            line += (vallist4[count2] + '')
                        segval = line
                        #print(inc2)
                        globals()[f'{p1}%s' % inc2] = segval
                        
        #entries = len(df) - 1
        #lines = len(HL7Field)
        ##print(lines)
        #if countalpha == entries:
        #    countbravo = 0
        #    for vals in varlist:
        #        #print(vals)
        #        #if 
        #        countbravo += 1
        #        print(eval(vals))
        #        if countbravo == lines:
        #            countbravo = 0
        #            print('\n')
        #    print('\n')
       
    #dfhl7RequiredFieldsMerge = dfhl7RequiredFieldsMerge.drop(columns=['HL7 Nat ELR Match','HL7 Element','Separator_y','HL7 Data Type','Data Type','Order', 'Field Name'])
    #display(dfhl7RequiredFieldsMerge)
    #dfforloopsfordays.to_excel(r'C:\Users\gmoye\FlatFileHL7\HL7LineData.xlsx')
    
#dfhl7RequiredFieldsMerge = dfhl7RequiredFieldsMerge.drop(columns = ['Field Name','HL7 Element Name','HL7 Element','Separator_y','HL7 Data Type','Data Type','Order'])

,Line Name,Line Code,Main Code,Sub Code,Segment,Separator_x,HL7 Nat ELR Match,HL7 Element,Required,Hardcode
0,DG1,1,1,0,DG1-1,|,Set ID - DG1,NaN,NaN,None
1,DG1,2,2,0,DG1-2,|,Diagnosis Coding Method,NaN,NaN,None
2,DG1,3.1,3,1,DG1-3.1,^,Identifier,NaN,NaN,None
3,DG1,3.2,3,2,DG1-3.2,^,Text,NaN,NaN,None
4,DG1,3.3,3,3,DG1-3.3,^,Name Of Coding System,NaN,NaN,None
5,DG1,3.4,3,4,DG1-3.4,^,Alternate Identifier,NaN,NaN,None
6,DG1,3.5,3,5,DG1-3.5,^,Alternate Text,NaN,NaN,None
7,DG1,3.6,3,6,DG1-3.6,^,Name Of Alternate Coding System,NaN,NaN,None
8,DG1,3,3,0,DG1-3,|,Diagnosis Code - DG1,NaN,NaN,None
9,DG1,4,4,0,DG1-4,|,Diagnosis Description,NaN,NaN,None


In [6]:
#dfhl7RequiredFieldsMergeTest = dfhl7RequiredFieldsMerge.drop(columns=['HL7 Nat ELR Match','HL7 Element','Separator_y','HL7 Data Type','Data Type','Order', 'Field Name'])
#display(dfhl7RequiredFieldsMergeTest)
#dfforloopsfordays.to_excel(r'C:\Users\gmoye\FlatFileHL7\HL7LineData.xlsx')

#dfforloopsfordayshc = dfhl7RequiredFieldsMerge.loc[(dfhl7RequiredFieldsMerge['Hardcode'].isna() == False) | (dfhl7RequiredFieldsMerge['HL7 Nat ELR Match'].isna() == False)]
#dfforloopsfordayshc['HL7 Nat ELR Match'] = dfforloopsfordayshc['HL7 Nat ELR Match'].fillna(dfforloopsfordayshc['Field Name'])
#display(dfforloopsfordayshc, len(dfforloopsfordayshc))

In [2]:
#count = 0
#p1 = 'linesegment'
#varlist = []
#countalpha = -1
#for i in range(len(df)):
#    countalpha += 1
#    strcountalpha = str(countalpha)
#    count4 = 0
#    for val in HL7Field:
#        linesegment = val
#        inc2 = val+'_'+strcountalpha
#        segval = str(p1 + inc2)
#        varlist.append(str(segval))
#        if val != '0009':
#            line = f'{val}' + "|"
#        else:
#            line = f'{val}'
#        count += 1
#        count1 = 0
#        for field, val1 in dfhl7RequiredFieldsMerge[['HL7 Element','Segment','HL7 Nat ELR Match']].items():
#            count1 += 1
#            if count1 == 2:
#                count2 = -1
#                for val2 in val1:
#                    #print(count2)
#                    count2 += 1
#                    for value in dfhl7RequiredFieldsMerge['HL7 Nat ELR Match'].items():
#                        f = dfhl7RequiredFieldsMerge['HL7 Nat ELR Match'].values[count2]
#                        #print(f, val2)
#                        break
#                    val2 = str(val2)
#                    if val in val2:
#                        try:
#                            #print(val2, val)
#                            Hardcode = dfhl7pivotreduction['Hardcode'].loc[(dfhl7pivotreduction.eq(f'{val2}').any(axis=1)) & (dfhl7pivotreduction['Hardcode'] != '|')]
#                            #Hardcode = str(Hardcode)
#                            #display(Hardcode)
#                            Hardcode = Hardcode.item()
#                            if count2 == 1:
#                                if Hardcode != []:
#                                    line += (Hardcode + '')
#                                else:
#                                    continue
#                            if count2 > 2:
#                                if count2 in [754,760,915,1108,1121,1758]:
#                                    pass
#                                #print(field)
#                                else:
#                                    if Hardcode != []:
#                                        line += (Hardcode + vallist4[count2] + '')
#                                    else:
#                                        continue
#                            else:
#                                continue
#                        except:
#                            pass
#                        
#                        count4 = -1
#                        count5 = 0
#                        for i in range(len(df)):
#                            count4 += 1
#                            for fieldval in df.columns:
#                                #count4 += 1
#                                fieldval = str(fieldval)
#                                f = str(f)
#                                for i in range(len(df)):
#                                    #count4 += 1
#                                    #print(i)
#                                    if f == fieldval:
#                                        #print(inc,count5)
#                                        v = df.loc[countalpha, fieldval]
#                                        v = str(v)
#                                        #print(count4, fieldval, f': {v}', val, val2)
#                                        line += (vallist4[count2] + v + '')
#                                    break
#                                count5 += 1
#                            break
#                            #break
#                        line += (vallist4[count2] + '')
#                    segval = line
#                    #print(inc2)
#                    globals()[f'{p1}%s' % inc2] = segval
 
entries = len(df) - 1
lines = len(HL7Field)
#print(lines)
if countalpha == entries:
    countbravo = 0
    for vals in varlist:
        #print(vals)
        #if 
        countbravo += 1
        print(eval(vals))
        if countbravo == lines:
            countbravo = 0
            print('\n')
    print('\n')

MSH|^~\&||^^^CHI.EPIC||^0DSomeCLIA^^^|CHI ST ALEXIUS HEALTH - TURTLE LAKE [79]|^^^Maven^2.16.840.1.114222.4.3.4.34.1.1^ISO||^^^NDHHS^2.16.840.1.113883.3.89.109.100.1.3^ISO||^^|2022-11-17 15:36:21||^^^ORU^R01^ORU_R01|||^^P||^^^2.5.1||||NE||NE||||^^^^^^||^^^^PHLabReport-NoAck^HL7^2.16.840.1.113883.9.11^ISO||
PID||^N9171279^^^^EPIC.ACC^^^^^^^|^^^^^^^^^^|^800390658^^^^EPIC.MRN^^^^^^^|^Ravnaas^^Carol^^Marlene^^nan^^^^^^^^^^^|^^^^^^^^^^^^^^|^^||^^^^^^^^^^^^^^|^^White or Caucasian^^^^^|White or Caucasian|^PO Box 192^^^TURTLE LAKE^^ND^^58575^^^^^1110^^^^^^||^701-426-9366^^^^^^^^^^^^|^^^^^^^^^^^^|^^^^^^|^^^^^^|^^^^^^|^^^^^^^^^^||^^^|^^^^^^^^^^|^^Non-Hispanic, Latino/a, Latinx [1]^^^^^||||^^^^^^|^^^^^^|^^^^^^|^^||||^^|^^^|^^^^^^|^^^^^^||^^^^^^|^^^^^^^^^|
ORC|RE||^^^^|^^^^|^^^^|||^^^^^^^^^^^^|^^|^^|^^^^^^^^^^^^^^^^^^^^^^^|^^^^^^^^^^^^^^^^^^^^^^^|^1649839846^^^^^^^^^^^^^^NORTHERN PLAINS LABORATORY, LLC [78]^^^^^^^^^^|^^^^^^^^^^^|^701-462-3396^^^^^^^^^^^^|^^|^^^^^^|^^^^^^|^^^^^^|^^^^^^^^^^^^^^^^^^^

In [234]:
display(df)

#for vals in varlist:
#    #print(vals)
#    countbravo += 1
#    print(eval(vals))
#    if countbravo == lines:
#        countbravo = 0
#        print('\n')
#print('\n')

,Order_test_date,Test_date,Specimen_collection_date_time,Testing_lab_name,Reporting_facility_name,Patient_ID,Patient_ID_2,Patient_last_name,Patient_first_name,Patient_middle_name,Patient_suffix,Patient_race,Patient_DOB,Patient_gender,Patient_ethnicity,Patient_street,Street Address.1,Patient_city,Patient_county,Patient_state,Patient_zip_code,Patient_phone_number,Ordering_provider_ID,Ordering_provider_first_name,Ordering_provider_street,Ordering_provider_street2,Ordering_provider_city,Ordering_provider_state,Ordering_provider_zip_code,Ordering_provider_phone,Testing_lab_specimen_ID,Procedure,Ordered_test_code,Test_performed_number,Test_performed_description,Last Lab Results,Test_result_description,Specimen_type_code,Specimen_source_site_code,Comments,Patient_occupation,Employment Status,Patient Employer,Ambulatory Status,Patient_ID_assigner,Patient_ID_2_assigner
Field,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2022-11-17 15:36:21,2022-11-17 15:59:12,2022-11-17 15:51:00,CHI ST ALEXIUS HEALTH - TURTLE LAKE [79],"NORTHERN PLAINS LABORATORY, LLC [78]",N9171279,800390658,Ravnaas,Carol,Marlene,NaN,White or Caucasian,1939-04-11,Female,"Non-Hispanic, Latino/a, Latinx [1]",PO Box 192,NaN,TURTLE LAKE,1110,ND,58575,701-426-9366,1649839846,Kaitlyn R Wirtz,1177 Border Lane,NaN,Washburn,ND,58577,701-462-3396,347306692,"SARS COV2 (TL, GAR. MIN & MAN ONLY) [LAB439701]",[94558-4],1,"SARS COV2 (TL, GAR. MIN & MAN ONLY)",'SARS CoV Ag: Negative','Negative',Nares,Nares,The SARS CoV Ag test has been performed under ...,NaN,Retired [5],NaN,No,EPIC.ACC,EPIC.MRN
1,2022-09-05 10:20:52,2022-09-05 10:32:44,2022-09-05 10:15:00,CHI ST ALEXIUS HEALTH - TURTLE LAKE [79],"NORTHERN PLAINS LABORATORY, LLC [78]",N7050230,800535295,Baker,John,E,NaN,White or Caucasian,1953-04-17,Male,"Non-Hispanic, Latino/a, Latinx [1]",2ND AVE W,NaN,SAWYER,1812,ND,58781,701-509-4135,1235213695,Susan Gale Wardner,220 5th Ave W,NaN,TURTLE LAKE,ND,58575,701-448-2331,337986190,"SARS COV2 (TL, GAR. MIN & MAN ONLY) [LAB439701]",[94558-4],1,"SARS COV2 (TL, GAR. MIN & MAN ONLY)",'SARS CoV Ag: Positive','Positive',Nares,Nares,The SARS CoV Ag test has been performed under ...,NaN,Retired [5],NaN,No,EPIC.ACC,EPIC.MRN
2,2022-06-21 08:16:18,2022-06-21 08:50:56,2022-06-21 08:20:00,CHI ST ALEXIUS HEALTH - TURTLE LAKE [79],"NORTHERN PLAINS LABORATORY, LLC [78]",N4210361,800571449,Lemke,William,August,Jr.,White or Caucasian,1951-07-09,Male,"Non-Hispanic, Latino/a, Latinx [1]",PO Box 313,NaN,TURTLE LAKE,1110,ND,58575,701-448-2082,1235213695,Susan Gale Wardner,220 5th Ave W,NaN,TURTLE LAKE,ND,58575,701-448-2331,328550952,"SARS COV2 (TL, GAR. MIN & MAN ONLY) [LAB439701]",[94558-4],1,"SARS COV2 (TL, GAR. MIN & MAN ONLY)",'SARS CoV Ag: Negative','Negative',Nares,Nares,The SARS CoV Ag test has been performed under ...,NaN,Retired [5],NaN,No,EPIC.ACC,EPIC.MRN
3,2022-09-06 08:32:19,2022-09-06 08:45:42,2022-09-06 08:41:00,CHI ST ALEXIUS HEALTH - TURTLE LAKE [79],"NORTHERN PLAINS LABORATORY, LLC [78]",N7060385,800571449,Lemke,William,August,Jr.,White or Caucasian,1951-07-09,Male,"Non-Hispanic, Latino/a, Latinx [1]",PO Box 313,NaN,TURTLE LAKE,1110,ND,58575,701-448-2082,1235213695,Susan Gale Wardner,220 5th Ave W,NaN,TURTLE LAKE,ND,58575,701-448-2331,338673451,"SARS COV2 (TL, GAR. MIN & MAN ONLY) [LAB439701]",[94558-4],1,"SARS COV2 (TL, GAR. MIN & MAN ONLY)",'SARS CoV Ag: Positive','Positive',Nares,Nares,The SARS CoV Ag test has been performed under ...,NaN,Retired [5],NaN,No,EPIC.ACC,EPIC.MRN
4,2022-06-03 10:17:47,2022-06-03 10:26:37,2022-06-03 10:10:00,CHI ST ALEXIUS HEALTH - TURTLE LAKE [79],"NORTHERN PLAINS LABORATORY, LLC [78]",N4030558,800673696,Boyko,Maynard,NaN,NaN,White or Caucasian,1924-12-15,Male,"Non-Hispanic, Latino/a, Latinx [1]",1955 22nd St NW,NaN,RUSO,1110,ND,58778,701-679-2440,1235213695,Susan Gale Wardner,220 5th Ave W,NaN,TURTLE LAKE,ND,58575,701-448-2331,326122203,"SARS COV2 (TL, GAR. MIN & MAN ONLY) [LAB439701]",[94558-4],1,"SARS COV2 (TL, GAR. MIN & MAN ONLY)",'SARS CoV Ag: Positive

In [232]:
#inc2 = countalpha - 1
#inc2 = str(inc2)
#print(val+'-'+inc2)
display(varlist)
print(lines)

['linesegmentMSH_0',
 'linesegmentPID_0',
 'linesegmentORC_0',
 'linesegmentOBR_0',
 'linesegmentOBX_0',
 'linesegmentSPM_0',
 'linesegmentNTE_0',
 'linesegmentIN1_0',
 'linesegmentIN2_0',
 'linesegmentDG1_0',
 'linesegment0009_0',
 'linesegmentMSH_1',
 'linesegmentPID_1',
 'linesegmentORC_1',
 'linesegmentOBR_1',
 'linesegmentOBX_1',
 'linesegmentSPM_1',
 'linesegmentNTE_1',
 'linesegmentIN1_1',
 'linesegmentIN2_1',
 'linesegmentDG1_1',
 'linesegment0009_1',
 'linesegmentMSH_2',
 'linesegmentPID_2',
 'linesegmentORC_2',
 'linesegmentOBR_2',
 'linesegmentOBX_2',
 'linesegmentSPM_2',
 'linesegmentNTE_2',
 'linesegmentIN1_2',
 'linesegmentIN2_2',
 'linesegmentDG1_2',
 'linesegment0009_2',
 'linesegmentMSH_3',
 'linesegmentPID_3',
 'linesegmentORC_3',
 'linesegmentOBR_3',
 'linesegmentOBX_3',
 'linesegmentSPM_3',
 'linesegmentNTE_3',
 'linesegmentIN1_3',
 'linesegmentIN2_3',
 'linesegmentDG1_3',
 'linesegment0009_3',
 'linesegmentMSH_4',
 'linesegmentPID_4',
 'linesegmentORC_4',
 'lineseg

11


In [223]:
entries = len(df)
print(entries)

14


In [236]:
df.to_excel(r'C:\Users\gmoye\FlatFileHL7\ExtractTransformation.xlsx')

In [ ]:
display()